In [2]:
import pandas as pd
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [6]:
train_file = "./ChatbotData.csv"
data = pd.read_csv(train_file, delimiter=",")

#features = data["Q"]
#answers = data["A"]

lines = data.loc[:, 'Q':'A']
lines.sample(10)

,Q,A
8066,정말 희망 없는 건가?,미련 갖지 마세요.
2308,사회에서 좋은 사람 만날 수 있을까?,대하는 태도에 따라 다를 것 같아요.
1185,당구 쳤어,재밌겠어요.
5868,내가 나쁜 놈이지,당신은 좋은 사람이에요.
8121,조금 아픈가 보다 아주 쪼금,쪼그만 아파하고 괜찮아지세요.
1226,대학 간다고 달라질까,행복은 마음 가짐에 있어요.
5229,회사에 완전체가 있어,상대를 용서하면서 분출할 수 있는 탈출구를 찾아보세요.
11006,정말 잘 맞는 사람이 있을까?,잘 맞춰가는 사람이 있는 거겠죠.
9954,사랑하는 사람은 표시가 있었으면 좋겠다,당신이 그 신호를 못 찾은게 아닐까요.
3012,약초 캐볼까?,근처 산에 가보세요.


In [7]:
# text preprocessing
lines.A = lines.A.apply(lambda x:'\t '+x+' \n')
lines.sample(10)

,Q,A
3148,여름인데 태닝할까,\t 구릿빛 피부 좋죠! \n
3806,일 끝나면 그냥 쉬고 싶어,\t 운동으로 땀을 내보세요. 한결 몸이 가벼워질거예요. \n
6202,마음이 변하는 이유,\t 그동안 쌓인 것들이 원인이 아닐까 생각해요. \n
9385,남자친구와 동거 해도되는걸까?,\t 사랑한다면 선택할 수 있겠죠. \n
11693,친구들과의 모임이 너무 많은 사람,\t 적당하게 줄여주거나 나에게 믿음을 주라고 말해보세요. \n
1553,맘속으로 조금씩 정리가 돼가네,\t 다행이에요. \n
10506,어제 짝녀랑 데이트했는데 꿈만 같습니다.,\t 또 데이트할 수 있어요. \n
4151,제 2 외국어 뭐할까,\t 요즘은 잘 안 배우는 언어도 좋은 거 같아요. \n
11424,짝남이랑 동생 이름이 똑같아.,\t 놀랐겠어요. \n
4126,점심시간이라도 마음 놓고 쉬고 싶어,\t 맛난거 드시고 조금이라도 쉬셔야 할텐데. \n


In [9]:
# text combination

# Questions
q_vocab = set()
for line in lines.Q: # 1줄씩 읽음
    for char in line: # 1개의 문자씩 읽음
        q_vocab.add(char)

# Answers
a_vocab = set()
for line in lines.A:
    for char in line:
        a_vocab.add(char)

In [10]:
a_vocab_size = len(a_vocab)+1
q_vocab_size = len(q_vocab)+1
print("Answer 문장의 char 집합 : ", a_vocab_size)
print("Question 문장의 char 집합 : ", q_vocab_size)

Answer 문장의 char 집합 :  975
Question 문장의 char 집합 :  1152


In [12]:
a_vocab = sorted(list(a_vocab))
q_vocab = sorted(list(q_vocab))

print(a_vocab[45: 75])
print(q_vocab[45: 75])

['갯', '갱', '거', '걱', '건', '걷', '걸', '검', '겁', '것', '겉', '게', '겐', '겟', '겠', '겨', '격', '겪', '견', '결', '겼', '경', '곁', '계', '고', '곡', '곤', '곧', '골', '곰']
['갑', '값', '갔', '강', '갖', '같', '갚', '개', '객', '갠', '갱', '걍', '걔', '거', '걱', '건', '걷', '걸', '검', '겁', '것', '게', '겐', '겜', '겟', '겠', '겨', '격', '겪', '견']


In [13]:
a_to_index = dict([(word, i+1) for i, word in enumerate(a_vocab)])
q_to_index = dict([(word, i+1) for i, word in enumerate(q_vocab)])
print(a_to_index)
print(q_to_index)

{'\t': 1, '\n': 2, ' ': 3, '!': 4, "'": 5, ',': 6, '-': 7, '.': 8, '0': 9, '1': 10, '2': 11, '3': 12, '5': 13, '6': 14, '7': 15, '8': 16, '?': 17, 'A': 18, 'C': 19, 'N': 20, 'S': 21, 'X': 22, 'a': 23, 'g': 24, 'n': 25, 'o': 26, 's': 27, '~': 28, '…': 29, 'ㅊ': 30, 'ㅠ': 31, '가': 32, '각': 33, '간': 34, '갈': 35, '감': 36, '갑': 37, '값': 38, '갔': 39, '강': 40, '갖': 41, '같': 42, '갚': 43, '개': 44, '객': 45, '갯': 46, '갱': 47, '거': 48, '걱': 49, '건': 50, '걷': 51, '걸': 52, '검': 53, '겁': 54, '것': 55, '겉': 56, '게': 57, '겐': 58, '겟': 59, '겠': 60, '겨': 61, '격': 62, '겪': 63, '견': 64, '결': 65, '겼': 66, '경': 67, '곁': 68, '계': 69, '고': 70, '곡': 71, '곤': 72, '곧': 73, '골': 74, '곰': 75, '곱': 76, '곳': 77, '공': 78, '과': 79, '관': 80, '광': 81, '괘': 82, '괜': 83, '괴': 84, '교': 85, '구': 86, '국': 87, '군': 88, '굳': 89, '굴': 90, '굽': 91, '궁': 92, '궈': 93, '권': 94, '귀': 95, '귄': 96, '귈': 97, '규': 98, '균': 99, '귤': 100, '그': 101, '극': 102, '근': 103, '글': 104, '금': 105, '급': 106, '긋': 107, '긍': 108, '기': 109, '긴': 110, '길': 

In [17]:
encoder_input = []

# 1개의 문장
for line in lines.Q:
  encoded_line = []
  # 각 줄에서 1개의 char
  for char in line:
    # 각 char을 정수로 변환
    encoded_line.append(q_to_index[char])
  encoder_input.append(encoded_line)
print('source 문장의 정수 인코딩 :',encoder_input[:5])

source 문장의 정수 인코딩 : [[8, 9, 644, 1, 335, 2], [8, 859, 444, 1, 1083, 97, 1, 340, 709, 832, 709], [10, 493, 11, 793, 1, 224, 380, 40, 83, 1, 652, 252], [10, 493, 11, 793, 1, 823, 284, 1, 224, 380, 40, 83, 1, 652, 252], [26, 26, 23, 1, 649, 1082, 211]]


In [18]:
decoder_input = []
for line in lines.A:
  encoded_line = []
  for char in line:
    encoded_line.append(a_to_index[char])
  decoder_input.append(encoded_line)
print('target 문장의 정수 인코딩 :',decoder_input[:5])

target 문장의 정수 인코딩 : [[1, 3, 920, 359, 32, 3, 301, 3, 32, 183, 655, 8, 3, 2], [1, 3, 673, 351, 928, 3, 272, 379, 217, 223, 8, 3, 2], [1, 3, 628, 932, 678, 3, 615, 714, 159, 3, 726, 729, 8, 3, 2], [1, 3, 628, 932, 678, 3, 615, 714, 159, 3, 726, 729, 8, 3, 2], [1, 3, 204, 509, 683, 3, 767, 904, 342, 743, 729, 8, 3, 2]]


In [19]:
decoder_target = []
for line in lines.A:
  timestep = 0
  encoded_line = []
  for char in line:
    if timestep > 0:
      encoded_line.append(a_to_index[char])
    timestep = timestep + 1
  decoder_target.append(encoded_line)
print('target 문장 레이블의 정수 인코딩 :',decoder_target[:5])

target 문장 레이블의 정수 인코딩 : [[3, 920, 359, 32, 3, 301, 3, 32, 183, 655, 8, 3, 2], [3, 673, 351, 928, 3, 272, 379, 217, 223, 8, 3, 2], [3, 628, 932, 678, 3, 615, 714, 159, 3, 726, 729, 8, 3, 2], [3, 628, 932, 678, 3, 615, 714, 159, 3, 726, 729, 8, 3, 2], [3, 204, 509, 683, 3, 767, 904, 342, 743, 729, 8, 3, 2]]


In [20]:
max_A_len = max([len(line) for line in lines.A])
max_Q_len = max([len(line) for line in lines.Q])
print('source 문장의 최대 길이 :',max_A_len)
print('target 문장의 최대 길이 :',max_Q_len)

source 문장의 최대 길이 : 80
target 문장의 최대 길이 : 56


In [22]:
encoder_input = pad_sequences(encoder_input, maxlen=max_Q_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_A_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_A_len, padding='post')

encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [23]:
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import numpy as np

In [24]:
encoder_inputs = Input(shape=(None, q_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)

# encoder_outputs don't necessary
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)


# LSTM은 바닐라 RNN과 달리 상태가 두개 은닉 상태와 셀 상태
encoder_states = [state_h, state_c]

In [25]:
decoder_inputs = Input(shape=(None, a_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)

# 디코더에게 인코더의 은닉 상태, 셀 상태를 전달
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(a_vocab_size, activation="softmax")
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [29]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=20, epochs=20, validation_split=0.2)

ResourceExhaustedError: {{function_node __wrapped____EagerConst_device_/job:localhost/replica:0/task:0/device:GPU:0}} SameWorkerRecvDone unable to allocate output tensor. Key: /job:localhost/replica:0/task:0/device:CPU:0;249d306f74e3f936;/job:localhost/replica:0/task:0/device:GPU:0;edge_2_input;0:0